<a href="https://colab.research.google.com/github/Mharishankar1/CodSoft-ML/blob/main/Handwriting_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
3010/3010 [==============================] - 221s 72ms/step - loss: 6.2373 - accuracy: 0.0772
Epoch 2/20
3010/3010 [==============================] - 202s 67ms/step - loss: 5.5122 - accuracy: 0.1239
Epoch 3/20
3010/3010 [==============================] - 207s 69ms/step - loss: 5.1293 - accuracy: 0.1476
Epoch 4/20
3010/3010 [==============================] - 203s 68ms/step - loss: 4.7988 - accuracy: 0.1651
Epoch 5/20
3010/3010 [==============================] - 199s 66ms/step - loss: 4.4958 - accuracy: 0.1840
Epoch 6/20
3010/3010 [==============================] - 201s 67ms/step - loss: 4.2098 - accuracy: 0.2023
Epoch 7/20
3010/3010 [==============================] - 198s 66ms/step - loss: 3.9383 - accuracy: 0.2279
Epoch 8/20
3010/3010 [==============================] - 198s 66ms/step - loss: 3.6793 - accuracy: 0.2575
Epoch 9/20
3010/3010 [==============================] - 199s 66ms/step - loss: 3.4378 - accuracy: 0.2891
Epoch 10/20
3010/3010 [==============================] 

In [ ]:
seed_text = "This hotel is"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_liist), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 47ms/step
This hotel is what i found


In [ ]:
with open("output.txt",'w') as file:
  file.writelines([seed_text])
  file.close()

In [ ]:
from PIL import Image
with open('output.txt','r') as txt:
    lines = txt.readlines()

BG=Image.open('drive/MyDrive/myfont/bg.png')
for line in lines:
  text = line.strip()
  sheet_width=BG.width
  gap, ht = 0, 0

  for i in text.replace("\n"," "):
      cases = Image.open("drive/MyDrive/myfont/{}.png".format(str(ord(i))))
      BG.paste(cases, (gap, ht))
      size = cases.width
      height=cases.height
      gap+=size
      if sheet_width < gap or len(i)*115 >(sheet_width-gap):
          gap,ht=0,ht+140
  print(gap)
  print(sheet_width)
  BG.show()

1468
2632


In [ ]:
BG.save('handwritten_text.png')
